## NBA GM Trades Analysis - Part 1 Data Cleaning

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import requests
import time
import json
from bs4 import BeautifulSoup

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
years = list(range(2022,2025))

In [ ]:
winshares_df = pd.read_csv('../inputs/win_shares_stats_2022-2024.csv')

In [ ]:
trades_df = pd.read_csv('../inputs/NBA_GM_Transaction_Dataset_ALL_TEAMS_Trade_Data_2022-2024.csv')

### Merging Data

In [7]:
merged_df = pd.merge(trades_df, winshares_df[['Player', 'Year', 'Tm', 'WS']], 
                     left_on=['Acquired Player', 'Season', 'Team Acquiring'], 
                     right_on=['Player', 'Year', 'Tm'], 
                     how='left')

In [8]:
# Assign the WS values to Acquired Player BPM/WARP Year 1 column
trades_df['Acquired Player Win Shares Year 1'] = merged_df['WS']

In [9]:
trades_df[trades_df['Acquired Player'] == 'James Harden']

,Season,Team Acquiring,Acquired Player,Team Losing,Acquisition Type,Acquiring GM,Losing GM,Year 2,Helper,AAV,Acquired Player Win Shares Year 1
106,2022,PHI,James Harden,BRK,Trade,Daryl Morey,Sean Marks,2023,2022James Harden,44.3,3.3
204,2024,LAC,James Harden,PHI,Trade,Trent Redden,Daryl Morey,2025,2024James Harden,35.6,8.4


In [10]:
winshares_df[winshares_df['Player'] == 'James Harden']

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
292,James Harden,PG,32,TOT,65,2420,20.9,.583,.452,.540,2.6,19.7,11.3,40.3,1.7,1.3,18.8,27.2,4.9,2.7,7.6,.152,3.7,0.3,4.0,3.7,2022
293,James Harden,PG,32,BRK,44,1627,20.4,.576,.435,.496,2.9,20.2,11.7,39.7,1.7,1.7,19.8,28.3,2.6,1.8,4.4,.130,3.7,0.4,4.1,2.5,2022
294,James Harden,PG,32,PHI,21,792,21.8,.601,.493,.650,1.9,18.9,10.5,41.4,1.6,0.6,16.4,24.9,2.3,0.9,3.3,.197,3.8,0.1,3.8,1.2,2022
1096,James Harden,PG,33,PHI,58,2135,21.6,.607,.496,.429,2.3,17.1,9.8,43.3,1.6,1.4,16.3,25.0,5.8,2.6,8.4,.188,5.1,0.3,5.4,4.0,2023
1824,James Harden,PG,34,LAC,72,2470,18.6,.612,.594,.418,1.6,14.8,8.4,33.6,1.5,2.1,16.0,20.6,5.9,2.5,8.4,.163,3.8,0.3,4.1,3.8,2024


In [11]:
merged_df_year2 = pd.merge(trades_df, winshares_df[['Player', 'Year', 'Tm', 'WS']], 
                     left_on=['Acquired Player', 'Year 2', 'Team Acquiring'], 
                     right_on=['Player', 'Year', 'Tm'], 
                     how='left')

In [12]:
trades_df['Acquired Player Win Shares Year 2'] = merged_df_year2['WS']

The resulting dataset is all players acquired by trade from 2022-2024 and their performance in the subsequent seasons.

In [13]:
trades_df.isna().sum()

Season                                 0
Team Acquiring                         0
Acquired Player                        0
Team Losing                            0
Acquisition Type                       0
Acquiring GM                           0
Losing GM                              0
Year 2                                 0
Helper                                 0
AAV                                    6
Acquired Player Win Shares Year 1     83
Acquired Player Win Shares Year 2    151
dtype: int64

In [14]:
trades_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Season                             234 non-null    int64  
 1   Team Acquiring                     234 non-null    object 
 2   Acquired Player                    234 non-null    object 
 3   Team Losing                        234 non-null    object 
 4   Acquisition Type                   234 non-null    object 
 5   Acquiring GM                       234 non-null    object 
 6   Losing GM                          234 non-null    object 
 7   Year 2                             234 non-null    int64  
 8   Helper                             234 non-null    object 
 9   AAV                                228 non-null    float64
 10  Acquired Player Win Shares Year 1  151 non-null    object 
 11  Acquired Player Win Shares Year 2  83 non-null     object 

From here it looks most players (151/234) in this dataset did not stay on the same team in year 2 after the acquisition.

In [15]:
trades_df[trades_df['Acquired Player Win Shares Year 2'].notna()]

,Season,Team Acquiring,Acquired Player,Team Losing,Acquisition Type,Acquiring GM,Losing GM,Year 2,Helper,AAV,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2
0,2022,BOS,Al Horford,OKC,Trade,Brad Stevens,Sam Presti,2023,2022Al Horford,26.5,7.6,6.3
2,2022,LAC,Jason Preston,ORL,Trade,Michael Winger,John Hammond,2023,2022Jason Preston,1.0,NaN,0.1
5,2022,CHO,Kai Jones,NYK,Trade,Mitch Kupchak,Scott Perry,2023,2022Kai Jones,2.9,0.0,0.7
7,2022,NYK,Quentin Grimes,LAC,Trade,Scott Perry,Michael Winger,2023,2022Quentin Grimes,2.2,1.4,4.8
8,2022,NYK,Miles McBride,OKC,Trade,Scott Perry,Sam Presti,2023,2022Miles McBride,1.1,0.2,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...
190,2023,IND,Jordan Nwora,MIL,Trade,Chad Buchanan,Jon Horst,2024,2023Jordan Nwora,1.7,0.8,0.1
191,2023,MIL,Jae Crowder,PHO,Trade,Jon Horst,James Jones,2024,2023Jae Crowder,10.1,1.2,1.6
192,2023,PHO,Kevin Durant,BRK,Trade,James Jones,Sean Marks,2024,2023Kevin Durant,47.6,1.1,8.3
195,2023,NYK,Josh Hart,POR,Trade,Scott Perry,Joe Cronin,2024,2023Josh Hart,12.9,2.8,5.7


In [16]:
pd.set_option('display.max_rows', None)

In [17]:
trades_df[trades_df['Acquired Player Win Shares Year 1'].isna()].sort_values('AAV', ascending=False)

,Season,Team Acquiring,Acquired Player,Team Losing,Acquisition Type,Acquiring GM,Losing GM,Year 2,Helper,AAV,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2
111,2022,WAS,Kristaps Porziņģis,DAL,Trade,Tommy Sheppard,Nico Harrison,2023,2022Kristaps Porziņģis,33.8,NaN,NaN
105,2022,BRK,Ben Simmons,PHI,Trade,Sean Marks,Daryl Morey,2023,2022Ben Simmons,33.0,NaN,2.2
18,2022,TOR,Goran Dragic,MIA,Trade,Bobby Webster,Andy Elisburg,2023,2022Goran Dragic,19.4,NaN,NaN
152,2023,DET,Bojan Bogdanović,UTA,Trade,Troy Weaver,Justin Zanik,2024,2023Bojan Bogdanović,19.3,NaN,NaN
218,2024,CHO,Dāvis Bertāns,OKC,Trade,Mitch Kupchak,Sam Presti,2025,2024Dāvis Bertāns,17.0,NaN,NaN
85,2022,IND,Ricky Rubio,CLE,Trade,Chad Buchanan,Koby Altman,2023,2022Ricky Rubio,17.0,NaN,NaN
109,2022,DAL,Davis Bertans,WAS,Trade,Nico Harrison,Tommy Sheppard,2023,2022Davis Bertans,16.0,NaN,NaN
33,2022,NOP,Jonas Valanciunas,MEM,Trade,Trajan Langdon,Zach Kleiman,2023,2022Jonas Valanciunas,14.7,NaN,NaN
219,2024,DAL,P. J. Washington,CHO,Trade,Nico Harrison,Mitch Kupchak,2025,2024P. J. Washington,13.5,NaN,NaN
53,2022,MEM,Patrick Beverley,LAC,Trade,Zach Kleiman,Michael Winger,2023,2022Patrick Beverley,13.0,NaN,NaN


In [18]:
trades_df.loc[(trades_df['Acquired Player'] == 'Kristaps Porziņģis') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 2.2
trades_df.loc[(trades_df['Acquired Player'] == 'Ben Simmons') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 0
trades_df.loc[(trades_df['Acquired Player'] == 'Goran Dragic') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 0.3
trades_df.loc[(trades_df['Acquired Player'] == 'Bojan Bogdanović') & (trades_df['Season'] == 2023), 'Acquired Player Win Shares Year 1'] = 3.5
trades_df.loc[(trades_df['Acquired Player'] == 'Dāvis Bertāns') & (trades_df['Season'] == 2024), 'Acquired Player Win Shares Year 1'] = 0.4
trades_df.loc[(trades_df['Acquired Player'] == 'Davis Bertans') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 0.6
trades_df.loc[(trades_df['Acquired Player'] == 'Jonas Valanciunas') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 7.3
trades_df.loc[(trades_df['Acquired Player'] == 'P. J. Washington') & (trades_df['Season'] == 2024), 'Acquired Player Win Shares Year 1'] = 1.1
trades_df.loc[(trades_df['Acquired Player'] == 'Ricky Rubio') & (trades_df['Season'] == 2022), 'Acquired Player Win Shares Year 1'] = 1.3

Because some players have missing data, I have filled in the win shares data for relevant players above.

In [19]:
trades_df = trades_df.dropna(subset=['AAV'])

In [20]:
trades_df[trades_df['Acquired Player Win Shares Year 1'].isna()].sort_values('AAV', ascending=False)

,Season,Team Acquiring,Acquired Player,Team Losing,Acquisition Type,Acquiring GM,Losing GM,Year 2,Helper,AAV,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2
53,2022,MEM,Patrick Beverley,LAC,Trade,Zach Kleiman,Michael Winger,2023,2022Patrick Beverley,13.0,NaN,NaN
212,2024,HOU,Steven Adams,MEM,Trade,Rafael Stone,Zach Kleiman,2025,2024Steven Adams,12.6,NaN,NaN
206,2024,LAC,P. J. Tucker,PHI,Trade,Trent Redden,Daryl Morey,2025,2024P. J. Tucker,11.0,NaN,NaN
50,2022,SAS,Al-Farouq Aminu,CHI,Trade,Brian Wright,Marc Eversley,2023,2022Al-Farouq Aminu,10.2,NaN,NaN
157,2023,HOU,Derrick Favors,OKC,Trade,Rafael Stone,Sam Presti,2024,2023Derrick Favors,10.1,NaN,NaN
203,2024,HOU,Victor Oladipo,OKC,Trade,Rafael Stone,Sam Presti,2025,2024Victor Oladipo,9.5,NaN,NaN
142,2023,DET,Kemba Walker,NYK,Trade,Troy Weaver,Scott Perry,2024,2023Kemba Walker,9.2,NaN,NaN
1,2022,OKC,Kemba Walker,BOS,Trade,Sam Presti,Brad Stevens,2023,2022Kemba Walker,9.2,NaN,NaN
141,2023,WAS,Monté Morris,DEN,Trade,Will Dawkins,Calvin Booth,2024,2023Monté Morris,9.1,NaN,NaN
13,2022,MIN,Tauren Prince,CLE,Trade,Sachin Gupta,Koby Altman,2023,2022Tauren Prince,7.3,NaN,NaN


In [21]:
trades_df['Acquired Player Win Shares Year 1'] = trades_df['Acquired Player Win Shares Year 1'].fillna(0)

In [ ]:
trades_df.to_csv('../outputs/trade_acquired_win_shares_2022-2024.csv', index=False)